In [ ]:
import scipy.stats
import numpy as np
import matplotlib.pyplot as plt
from  scipy.stats import norm
from  scipy.stats import binom
from scipy.stats import beta
from  scipy.stats import uniform
from scipy.stats import gaussian_kde

import pystan
from pystan.constants import MAX_UINT

%run ./tools.py

# Exercise A

The goal of this exercise is to implement the Metropolis-Hastings method as seen in Lesson 5. Consider a variable y_i representing the number of success of an experiment i repeated n times. Now consider a vector Y containing the number of success of N independent experiments y_i. We assume that each experiment_i share a common probability of success $\theta = 0.3$. Therefore we can generate the following data:

In [ ]:
N = 100
n = 100
theta = 0.3

Y = binom.rvs(n, theta, size=N)
    
print(Y[:10])

**1) We will assume a prior Beta distribution with parameters a=0.5 and b=0.5 for $\theta$. Write the data likelihood p(Y|$\theta$) and the posterior distribution p($\theta$ | Y).**

**2) Estimate the posterior distirbution p($\theta$|Y) using the Metropolis-Hastings algorithm.**

In [ ]:
def proposal(prop_mu, prop_sigma):
    return scipy.stats.norm.rvs(loc = prop_mu, scale = prop_sigma, size = 1)

def log_likelihood(param):
    return np.sum([binom.logpmf(y_i,n,param) for y_i in Y])

def log_prior(param):
    return beta.logpdf(param, 0.5, 0.5)

def log_posterior(param):
    return log_likelihood(param) + log_prior(param)

**3) Plot the posterior probability density functions obtained with the MCMC approximation and the analytical solution.**

# Exercise B

This year a farmer ran an experiment  where he gave 10 cows medicine A and 10 medicine B and then measured whether they got sick (0) or not (1) during the summer season. Here is the resulting data:

In [ ]:
cowA = np.array([0, 1, 0, 0, 0, 0, 1, 0, 0, 0])
cowB = np.array([0, 0, 1, 1, 1, 0, 1, 1, 1, 0])

**1) The farmer wants to know: How effective are the drugs? What is the evidence that medicine A is better or worse than medicine B ?**

The outcome for each cow taking medecine A can be modeled as a random variable following a Bernouilli distribution with parameter $\theta_1$. Similarly, the outcome for each cow taking medecine B can be modeled as a random variable following a Bernouilli distribution with parameter $\theta_2$. Let's assume a unifrom prior for both $\theta_1$ and $\theta_2$. Our goal here is to obtain a posterior distribution of these two parameters. In this exercise, we propose to do it through the HMC method using the Pystan package.

**2)** Earlier this year the farmer  ran an experiment where he gave 10 cows a special diet that could increase the milk production. He recorded the number of liters of milk from these “diet” cows and from 15 “normal” cows during one month. This is the data:

In [ ]:
diet_milk = np.array([651., 679., 374., 601., 401., 609., 767., 709., 704., 679.])
normal_milk = np.array([798., 1139., 529., 609., 553., 743., 151., 544., 488., 555., 257., 692., 678., 675., 538.])

**2a)** The farmer now wants to know: Was the diet any good, does it results in better milk production? 

The most common approach here would be to model the milk production of each cow as a normal distribution. For the cows following a diet, their milk production can be modeled as $y_i \sim \mathcal{N}(\mu_{diet}, \sigma^{2}_{diet})$, while for normal cows the milk production can be modeled as $y_i \sim \mathcal{N}(\mu_{normal}, \sigma^{2}_{normal})$.
We should also add priors for these four parameters. A lazy option here could be to assign them uniform priors.

**3)** The farmer also has chickens. He tries different diets on them too with the hope that they will produce more eggs. Below is the number of eggs produced in one week by chickens on a diet and chickens eating normal chicken food:

In [ ]:
diet_eggs = np.array([6, 4, 2, 3, 4, 3, 0, 4, 0, 6, 3])
normal_eggs =  np.array([4, 2, 1, 1, 2, 1, 2, 1, 3, 2, 1])

**3a)** The farmer now wants to know: Was the diet any good, does it result in the chickens producing more eggs ? 

In this case we want to model count data, therefore a reasonable choice to model the number of eggs produced by the chickens would be a Poisson distribution. This distribution has one parameter $\lambda$ which stands for the mean count. 

**4)** The farmer is now wondering whether the amount of time a cow spends outside in the sunshine affects how much milk she produces. To test this he makes a controlled experiment where he picks out 20 cows and assigns to each of them a number of hours she should spend outside each day. The experiment runs for a month and the farmer records the number of liters of milk each cow produces. The data is the following:

In [ ]:
milk = np.array([685, 691, 476, 1151, 879, 725, 1190, 1107, 809, 539, 298, 805, 820, 498, 1026, 1217, 1177, 684, 1061, 834])

hours = np.array([3, 7, 6, 10, 6, 5, 10, 11, 9, 3, 6, 6, 3, 5, 8, 11, 12, 9, 5, 5])


**4a)**  Using this data on hours of sunshine and resulting liters of milk the farmer wants to know: Does sunshine affect milk production positively or negatively?
